In [1]:
import json
import os
import pandas as pd
import requests
import ssl
import sys
import urllib
from configparser import ConfigParser
from slugify import slugify


In [2]:
# Specify inputs
parser = ConfigParser()
parser.read('config.ini')

# Program state: either looking for existing
USE_EXISTING_LOOKUP = False
if (parser['Downloading'] == "True"):
    USE_EXISTING_LOOKUP = True   

# Main working directory
MAIN_DIR = parser['Downloading']['ParentWorkingDirectory']

# Where the files will be downloaded to
DATA_DIR = MAIN_DIR + parser['Downloading']['JSOutputDirectory']

# If generating or looking up a table
LOOKUP_DIR = MAIN_DIR + parser['Downloading']['LookupDirectory']
LOOKUP_FILE = parser['Downloading']['LookupFile']

# If generating a new dataset
if (bool(parser['Downloading']['ParquetDataset'])):
    PARQUET_FILE = MAIN_DIR + parser['Downloading']['ParquetDataset']      # running over sample data

In [3]:
def shorten_name(url_name):
    shortened_url = url_name.replace('https://', '').replace('http://', '').replace('/', '_')
    shortened_url = slugify(shortened_url)[:250]
    suffix = '.txt'
    file_name = shortened_url + suffix
    
    return file_name

In [4]:
if (USE_EXISTING_LOOKUP & os.path.isfile(LOOKUP_DIR + LOOKUP_FILE)):
    print("Existing Lookup table found...")
    url_data = pd.read_csv(LOOKUP_DIR + LOOKUP_FILE)
    
else:
    print("Generating Lookup table...")
    raw_data = pd.read_parquet(PARQUET_FILE, engine='pyarrow')
    url_data = pd.DataFrame()
    url_data['url'] = raw_data['script_url'].unique()
    url_data['filename'] = url_data['url'].apply(shorten_name)
    url_data = url_data.sort_values('filename')
    url_data['status'] = -1



Generating Lookup table...


In [6]:
url_data

,url,filename,status
118,http://a.visualrevenue.com/vrs.js,a-visualrevenue-com-vrs-js.txt,-1
97,https://a1.alicdn.com/creation/html/2016/06/20...,a1-alicdn-com-creation-html-2016-06-20-creatio...,-1
426,https://acc.alicdn.com/tfscom/TB1XEs7RXXXXXXuX...,acc-alicdn-com-tfscom-tb1xes7rxxxxxxuxxxxxxxxx...,-1
443,https://ad.doubleclick.net/ddm/adi/N46101.2990...,ad-doubleclick-net-ddm-adi-n46101-2990edmunds-...,-1
13,http://ads.pubmatic.com/AdServer/js/showad.js#...,ads-pubmatic-com-adserver-js-showad-js-pix-kdn...,-1
72,https://ae.nflximg.net/containers/prod/300x250...,ae-nflximg-net-containers-prod-300x250-monet-m...,-1
18,http://ajax.googleapis.com/ajax/libs/swfobject...,ajax-googleapis-com-ajax-libs-swfobject-2-2-sw...,-1
1,https://ajax.googleapis.com/ajax/libs/webfont/...,ajax-googleapis-com-ajax-libs-webfont-1-6-26-w...,-1
167,https://api.moxielinks.com/perform.php?action=...,api-moxielinks-com-perform-php-action-category...,-1
27,https://apis.google.com/js/plusone.js,apis-google-com-js-plusone-js.txt,-1


In [7]:
#for url_name in df['script_url'].unique():
#    file_name = shorten_name(DATA_DIR, url_name)
#    
#    #with open(file_name, 'w') as source_file:

#    try:
#        r = requests.get(url_name, verify=False)
#        print("{}\n\t\t{}".format(url_name, r.status_code))
#    except (requests.exceptions.RequestException) as e:
#        print("{}\n\t\t{}".format(url_name, e))
        

In [26]:
ssl._create_default_https_context = ssl._create_unverified_context
failed = []

for url_name in url_data['script_url']:
    
    
    with open(file_name, 'w') as source_file:
        try:
            source_file.write(urllib.request.urlopen(url_name).read().decode(
                'utf-8', 'backslashreplace'))
            print(url_name)
            print("")

        except (urllib.error.URLError, ValueError) as e:
            failed.append(url_name)
            print('Attempted:', url_name)
            print(str(e), '\n')

https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com

https://ajax.googleapis.com/ajax/libs/webfont/1.6.26/webfont.js

http://cpro.baidustatic.com/cpro/ui/noexpire/js/4.0.1/adClosefeedbackUpgrade.min.js

https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=fe1ad16a94c816&origin=http%3A%2F%2Farabi21.com

https://static.dynamicyield.com/scripts/12290/dy-coll-min.js

https://www.syracuse.edu/about/

https://www.googletagmanager.com/gtm.js?id=GTM-5FC97GL

https://www.syracuse.edu/wp-includes/js/wp-emoji-release.min.js?ver=4.9.1

https://www.google-analytics.com/analytics.js

https://code.jquery.com/jquery-migrate-1.4.1.min.js

https://www.syracuse.edu/wp-content/themes/g6-carbon/js/carbon-all.js?ver=6.3.6

https://www.syracuse.edu/wp-includes/js/wp-embed.min.js?ver=4.9.1

https://syr-piwik-prod.syr.edu/piwik.js

http://ads.pubmatic.com/AdServer/js/showad.js#PIX&kdntuid=1&SPu

https://cdn.krxd.net/controltag/rkrioms3o.js?advertiserid=5094938;campaignid=20560416;adid=410398615;siteid=2445008;placementid=210642386;creativeid=95827831

https://cdn.krxd.net/ctjs/controltag.js.c3e8e6311e44dfc4f051e4a261784fa1

https://c.iogous.com/js/banner/DIGITALICE_REC.js

https://b97.yahoo.co.jp/pagead/conversion_async.js

https://rsv.dga.jp/s/click_sec/new_i_search_assist.js

http://wfpscripts.webspectator.com/adblocker/blockadblock.js

https://cpro.baidustatic.com/sync.htm?cproid=0FF010C18A9DD8D15614FA2FE581A5F3%3AFG%3D1

https://static.criteo.net/js/ld/ld.js

https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f115af0c85ff058&origin=https%3A%2F%2Ffishki.net

http://atir.aipai.com/attachment/jslib/jquery-1.8.3.min.js

http://mlog.aipai.com/apweb.min.js



KeyboardInterrupt: 

In [11]:
from pathlib import Path
PARQUET_DIR = Path('/mnt/Data/UCOSP_DATA/resources/trial_1_fulldata/')
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in PARQUET_DIR.glob('*.parquet')
)


In [59]:
from pyspark.sql import SparkSession, functions, types
spark = SparkSession.builder.appName('URL getter').getOrCreate()
data = spark.read.parquet('/mnt/Data/UCOSP_DATA/resources/full_url_list_v2/*')
#data = spark.read.parquet('/mnt/Data/UCOSP_DATA/resources/sample_full_url_list/*')

In [60]:
data.count()

1253168